In [1]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer,Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq,EarlyStoppingCallback
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
import numpy as np
from datasets import Dataset

In [2]:
import pandas as pd
data = pd.read_csv("/kaggle/input/english-nagamese-raw-data/raw_data.csv")
print(data.head())

                                             English  \
0  Paul, Silvanus, and Timothy to the church of t...   
1  We always give thanks to God for all of you as...   
2  We remember before our God and Father your wor...   
3  Brothers loved by God, we know he has chosen you,   
4  because our gospel came to you not in word onl...   

                                            Nagamese  
0  Paul aru Silvanus aru Timothy pora Isor aru Pr...  
1  Amikhan hodai apnikhan nimite Isor ke dhanyaba...  
2  Amikhan pora apni khan laga biswas laga kaam, ...  
3  Isor pora morom kora bhai khan, amikhan jane T...  
4  kilemane Isor laga kotha apni khan logote khal...  


In [3]:
# data=data[:100]

In [4]:
import torch
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

# Load model and tokenizer
model_name = "facebook/m2m100_418M"
model = M2M100ForConditionalGeneration.from_pretrained(model_name)
tokenizer = M2M100Tokenizer.from_pretrained(model_name)



config.json:   0%|          | 0.00/908 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.71M [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

In [5]:
source_texts = list(data["Nagamese"])  
target_texts = list(data["English"]) 


# Split the data into training and validation sets
train_source_texts, val_source_texts, train_target_texts, val_target_texts = train_test_split(
    source_texts, target_texts, test_size=0.2, random_state=50
)


print(type(train_source_texts))
print(type(train_target_texts))
print(type(val_source_texts))
print(type(val_target_texts))


<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>


In [6]:
# Tokenize function
def tokenize_function(examples):
    model_inputs = tokenizer(examples["source_texts"], max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(examples["target_texts"], max_length=128, truncation=True, padding="max_length")["input_ids"]
    
    # Replace padding token ID with -100 to ignore padding loss
    labels = [[-100 if token == tokenizer.pad_token_id else token for token in label] for label in labels]
    
    model_inputs["labels"] = labels
    return model_inputs

train_data = {"source_texts": train_source_texts, "target_texts": train_target_texts}
val_data = {"source_texts": val_source_texts, "target_texts": val_target_texts}

train_dataset = Dataset.from_dict(train_data)
val_dataset = Dataset.from_dict(val_data)

train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/6360 [00:00<?, ? examples/s]

Map:   0%|          | 0/1590 [00:00<?, ? examples/s]

In [7]:
# Fine-tuning loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, EarlyStoppingCallback

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="steps", 
    save_strategy="steps",
    eval_steps=40,  
    save_steps=40, 
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,  
    warmup_steps=50,  
    seed=40,
    learning_rate=3e-5,  
    weight_decay=0.01,
    save_total_limit=6,
    num_train_epochs=30,  
    predict_with_generate=True,
    optim="adafactor", 
    report_to=[],
    fp16=True,
    logging_dir="./logs",
    logging_steps=40,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

trainer.train()

<ipython-input-7-36180f7f912f>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss,Validation Loss
40,3.559300,3.003683
80,2.916500,2.489897
120,2.510400,2.214893
160,2.340800,2.078492
200,2.246400,1.966220
240,2.102100,1.910542
280,2.042600,1.852030
320,2.011000,1.787067
360,1.987200,1.768263
400,1.902200,1.736725


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=1280, training_loss=1.7479784697294236, metrics={'train_runtime': 2664.3811, 'train_samples_per_second': 71.611, 'train_steps_per_second': 4.481, 'total_flos': 5541286467600384.0, 'train_loss': 1.7479784697294236, 'epoch': 3.2160804020100504})

In [8]:
model.save_pretrained("./m2m100-finetuned-nagamese-english-final")
tokenizer.save_pretrained("./m2m100-finetuned-nagamese-english-final")

('./m2m100-finetuned-nagamese-english-final/tokenizer_config.json',
 './m2m100-finetuned-nagamese-english-final/special_tokens_map.json',
 'm2m100-finetuned-nagamese-english-final/vocab.json',
 'm2m100-finetuned-nagamese-english-final/sentencepiece.bpe.model',
 './m2m100-finetuned-nagamese-english-final/added_tokens.json')

In [9]:
!pip install transformers sacrebleu
import os
import sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 44.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 124.5 kB/s eta 0:00:00


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [11]:
model = model.to(device)

In [12]:
source_texts = val_source_texts
reference_texts = val_target_texts

In [13]:
tokenizer.lang_code_to_id["nag"] = tokenizer.lang_code_to_id.get("eng") 

In [14]:
# Translate source texts using the model
model.eval()
machine_translations = []

for text in source_texts:
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["en"])
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    machine_translations.append(translated_text)
    

In [15]:
import sacrebleu

# Ensure reference_texts is in the required format
wrapped_reference_texts = [reference_texts]  

# Compute BLEU score
bleu = sacrebleu.corpus_bleu(machine_translations, wrapped_reference_texts)
print("BLEU Score:", bleu.score)

BLEU Score: 29.288698836224846
